Возьмите датасет с отключением клиентов от телефонной компании.
Подготовьте данные для решения задачи.
Решите задачу с помощью нейронной сети, используя слои Dense как мы делали на уроке.
Ваша задача правильно написать “голову” сети и подобрать правильно loss функцию.

In [233]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, precision_score, \
              recall_score, f1_score

from keras.models import Sequential
from keras.layers import Dense

In [234]:
#Прочитаем данные.
df_train = pd.read_csv("churn-bigml-80.csv")
df_test = pd.read_csv("churn-bigml-20.csv")

In [235]:
df_test

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,LA,117,408,No,No,0,184.5,97,31.37,351.6,80,29.89,215.8,90,9.71,8.7,4,2.35,1,False
1,IN,65,415,No,No,0,129.1,137,21.95,228.5,83,19.42,208.8,111,9.40,12.7,6,3.43,4,True
2,NY,161,415,No,No,0,332.9,67,56.59,317.8,97,27.01,160.6,128,7.23,5.4,9,1.46,4,True
3,SC,111,415,No,No,0,110.4,103,18.77,137.3,102,11.67,189.6,105,8.53,7.7,6,2.08,2,False
4,HI,49,510,No,No,0,119.3,117,20.28,215.1,109,18.28,178.7,90,8.04,11.1,1,3.00,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,WI,114,415,No,Yes,26,137.1,88,23.31,155.7,125,13.23,247.6,94,11.14,11.5,7,3.11,2,False
663,AL,106,408,No,Yes,29,83.6,131,14.21,203.9,131,17.33,229.5,73,10.33,8.1,3,2.19,1,False
664,VT,60,415,No,No,0,193.9,118,32.96,85.0,110,7.23,210.1,134,9.45,13.2,8,3.56,3,False
665,WV,159,415,No,No,0,169.8,114,28.87,197.7,105,16.80,193.7,82,8.72,11.6,4,3.13,1,False


In [236]:
#Дропнем ненужные колонки.
for df in df_train, df_test:
    df.drop(['State',
             'Number vmail messages', 
             'Total day minutes',
             'Total eve minutes',
             'Total night minutes',
             'Total intl minutes'], axis=1, inplace=True)

In [237]:
df_train

,Account length,Area code,International plan,Voice mail plan,Total day calls,Total day charge,Total eve calls,Total eve charge,Total night calls,Total night charge,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,No,Yes,110,45.07,99,16.78,91,11.01,3,2.70,1,False
1,107,415,No,Yes,123,27.47,103,16.62,103,11.45,3,3.70,1,False
2,137,415,No,No,114,41.38,110,10.30,104,7.32,5,3.29,0,False
3,84,408,Yes,No,71,50.90,88,5.26,89,8.86,7,1.78,2,False
4,75,415,Yes,No,113,28.34,122,12.61,121,8.41,3,2.73,3,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,79,415,No,No,98,22.90,68,16.12,128,9.96,5,3.19,2,False
2662,192,415,No,Yes,77,26.55,126,18.32,83,12.56,6,2.67,2,False
2663,68,415,No,No,57,39.29,55,13.04,123,8.61,4,2.59,3,False
2664,28,510,No,No,109,30.74,58,24.55,91,8.64,6,3.81,2,False


In [238]:
#Cформируем перечни колонок в зависимости от типов.
numeric_features = df_train.select_dtypes(include=['int', 'float']).columns
categorical_features = df_train.select_dtypes(include=['object']).columns
target = df_train.columns[-1]

In [239]:
#Обработаем колонку с таргетом.
for df in df_train, df_test:
    df["Churn"] = df["Churn"].map({True: 1, False: 0})

In [240]:
df_train

,Account length,Area code,International plan,Voice mail plan,Total day calls,Total day charge,Total eve calls,Total eve charge,Total night calls,Total night charge,Total intl calls,Total intl charge,Customer service calls,Churn
0,128,415,No,Yes,110,45.07,99,16.78,91,11.01,3,2.70,1,0
1,107,415,No,Yes,123,27.47,103,16.62,103,11.45,3,3.70,1,0
2,137,415,No,No,114,41.38,110,10.30,104,7.32,5,3.29,0,0
3,84,408,Yes,No,71,50.90,88,5.26,89,8.86,7,1.78,2,0
4,75,415,Yes,No,113,28.34,122,12.61,121,8.41,3,2.73,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2661,79,415,No,No,98,22.90,68,16.12,128,9.96,5,3.19,2,0
2662,192,415,No,Yes,77,26.55,126,18.32,83,12.56,6,2.67,2,0
2663,68,415,No,No,57,39.29,55,13.04,123,8.61,4,2.59,3,0
2664,28,510,No,No,109,30.74,58,24.55,91,8.64,6,3.81,2,0


In [241]:
#Сделаем пайплайн для предобработки данных.
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(sparse=False))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

In [242]:
#Сформируем входные и выходные данные для тренировки и теста.
x_train, y_train = df_train.iloc[:, :-1], df_train.iloc[:, -1:]
x_test, y_test = df_test.iloc[:, :-1], df_test.iloc[:, -1:]

In [243]:
#Обучим пайплайн.
x_train = preprocessor.fit_transform(x_train)

In [244]:
x_train

array([[ 0.69216332, -0.52781119,  0.48486775, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.16127783, -0.52781119,  1.13537472, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.91968567, -0.52781119,  0.68502374, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-0.82465237, -0.52781119, -2.16719912, ...,  0.        ,
         1.        ,  0.        ],
       [-1.83586283,  1.70679716,  0.43482875, ...,  0.        ,
         1.        ,  0.        ],
       [-0.6729708 , -0.52781119,  0.63498474, ...,  0.        ,
         0.        ,  1.        ]])

In [245]:
x_test = preprocessor.transform(x_test)

In [246]:
x_test

array([[ 0.41408044, -0.69246654, -0.16563922, ...,  0.        ,
         1.        ,  0.        ],
       [-0.90049316, -0.52781119,  1.83592069, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.52641195, -0.52781119, -1.66680915, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.02689446, -0.52781119,  0.88517973, ...,  0.        ,
         1.        ,  0.        ],
       [ 1.47585142, -0.52781119,  0.68502374, ...,  0.        ,
         1.        ,  0.        ],
       [ 2.10785796,  1.70679716,  0.23467276, ...,  1.        ,
         1.        ,  0.        ]])

In [274]:
#Соберем модель
model = Sequential()

model.add(Dense(150, activation="relu"))
model.add(Dense(50, activation="relu"))
#model.add(Dense(25, activation="relu"))
model.add(Dense(2, activation="sigmoid"))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [275]:
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data = (x_test, y_test))

Epoch 1/10
84/84 [==============================] - 1s 4ms/step - loss: 0.4001 - accuracy: 0.8481 - val_loss: 0.3327 - val_accuracy: 0.8591
Epoch 2/10
84/84 [==============================] - 0s 1ms/step - loss: 0.3096 - accuracy: 0.8747 - val_loss: 0.2872 - val_accuracy: 0.8786
Epoch 3/10
84/84 [==============================] - 0s 1ms/step - loss: 0.2635 - accuracy: 0.8965 - val_loss: 0.2429 - val_accuracy: 0.8981
Epoch 4/10
84/84 [==============================] - 0s 1ms/step - loss: 0.2361 - accuracy: 0.9115 - val_loss: 0.2274 - val_accuracy: 0.9055
Epoch 5/10
84/84 [==============================] - 0s 1ms/step - loss: 0.2190 - accuracy: 0.9212 - val_loss: 0.2170 - val_accuracy: 0.9145
Epoch 6/10
84/84 [==============================] - 0s 1ms/step - loss: 0.2061 - accuracy: 0.9295 - val_loss: 0.2146 - val_accuracy: 0.9235
Epoch 7/10
84/84 [==============================] - 0s 1ms/step - loss: 0.1941 - accuracy: 0.9359 - val_loss: 0.2134 - val_accuracy: 0.9175
Epoch 8/10
84/84 [==

In [276]:
predictions = model.predict(x_test)
pred = predictions.argmax(axis = 1)

21/21 [==============================] - 0s 343us/step


In [277]:
precision_score(y_test, pred), recall_score(y_test, pred), f1_score(y_test, pred)

(0.8048780487804879, 0.6947368421052632, 0.7457627118644067)

In [278]:
confusion_matrix(y_test, pred)

array([[556,  16],
       [ 29,  66]], dtype=int64)